In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.348781
   2    1.075576
   3    1.333069
b  1   -1.545430
   3   -0.219162
c  1   -0.088370
   2    0.106830
d  2    1.210096
   3    1.132609
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']

1   -1.545430
3   -0.219162
dtype: float64

In [5]:
data["b":'c']

b  1   -1.545430
   3   -0.219162
c  1   -0.088370
   2    0.106830
dtype: float64

In [6]:
data[['b','d']]

b  1   -1.545430
   3   -0.219162
d  2    1.210096
   3    1.132609
dtype: float64

In [7]:
# Selection is even possible from an “inner” level:
data.loc[:,2]

a    1.075576
c    0.106830
d    1.210096
dtype: float64

__Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. For example, you could rearrange the data into a DataFrame using its unstack method:__

In [8]:
data.unstack()

,1,2,3
a,0.348781,1.075576,1.333069
b,-1.545430,NaN,-0.219162
c,-0.088370,0.106830,NaN
d,NaN,1.210096,1.132609


In [9]:
# The inverse operation of unstack is stack:
data.unstack().stack()

a  1    0.348781
   2    1.075576
   3    1.333069
b  1   -1.545430
   3   -0.219162
c  1   -0.088370
   2    0.106830
d  2    1.210096
   3    1.132609
dtype: float64

### With a DataFrame, either axis can have a hierarchical index:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

__The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:__

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### Reordering and Sorting Levels

In [13]:
# levels interchanged (but the data is otherwise unaltered):
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [14]:
# sorts the data using only the values in a single level.
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [15]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [27]:
# frame.sum(level='key2') On older versions of pandas, this method also works:
frame.groupby(level=0, axis=1).sum()

state      Colorado  Ohio
key1 key2                
a    1            2     1
     2            5     7
b    1            8    13
     2           11    19

In [26]:
frame.groupby(level='color', axis=1).sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [28]:
frame.groupby(level=1, axis=0).sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [30]:
frame.groupby(level="key1", axis=0).sum()

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

### Indexing with a DataFrame’s columns

In [31]:
frame = pd.DataFrame({'a': range(7),
                      'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [32]:
# DataFrame’s set_index function will create a new DataFrame using one or more of its columns as the index:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [33]:
# By default the columns are removed from the DataFrame, though you can leave them in:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [34]:
# reset_index, on the other hand, does the opposite of set_index; the hierarchical
# index levels are moved into the columns:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
